In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import warnings
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
warnings.filterwarnings("ignore",category=DeprecationWarning)
%matplotlib inline

#https://na80.lightning.force.com/lightning/r/Report/00O1P000003mYjbUAE/view

/Users/emeryames/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [29]:
df = pd.read_csv('year_data.csv')
df.Description.head()

0    Chat transcript\n\n\n\t\n\t\t\n\t\t\tName: GAr...
1    Chat transcript\n\n\n\t\n\t\t\n\t\t\tName: Gen...
2    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: c...
3    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: c...
4    Chat transcript\n\n\n\t\n\t\t\n\t\t\tE-mail: e...
Name: Description, dtype: object

In [31]:
data = df.Description.values.tolist()
data = [re.sub('\S*@\S*\s?', "", sent) for sent in data]
data = [re.sub('\s+', " ", sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]
data = [re.sub('https:\S+',"", sent) for sent in data]
data = [re.sub('Chat transcript',"", sent) for sent in data]
data = [re.sub('BevSpot Support here! How can we help',"", sent) for sent in data]
data = [re.sub('Chat has been opened from',"", sent) for sent in data]
data = [re.sub('E-mail', "", sent) for sent in data]
data = [re.sub('..:..:.. ..', "", sent) for sent in data]
data = [re.sub('America\S+', "", sent) for sent in data]
data = [re.sub('\S+.com\S+', "", sent) for sent in data]
data = [re.sub('BevSpot', "", sent) for sent in data]

pprint(data[:1])

[' Name: GAry Hipko : Brad Mon, 01/04/21   Hi from  Support! Check out our '
 'help center at  or chat us here if you have any questions as you move around '
 'the software. GAry Hipko  my food account is locked. why? Brad  Hi, Gary let '
 'me look into this for you. Gary, I am seeing a delinquency on the food '
 'account. Your grace period ended on 12/10. I am going to see if I can talk '
 'to your account manager. Do you mind waiting a few minutes? GAry Hipko  ive '
 'been waiting 1 1/2 hours for a call back. whats a few more minutes... Brad  '
 'Gary, I am very sorry to hear that. Did you have an appointment already '
 'scheduled? GAry Hipko  no i left a message Brad  Okay. Let me see what I can '
 'do. Gary, are you still with me? GAry Hipko  yes Brad  I am going to send '
 'you a link to Susans calendar so you can schedule an appointment to speak '
 'with her. I am also going to temporarily unlock the account until you '
 'resolve the billing issue with her. GAry Hipko  thanks Bra

In [32]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data))

In [33]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [34]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [35]:
data_words_nostops = remove_stopwords(data)
data_words_bigrams = make_bigrams(data_words_nostops)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [36]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [37]:
#no touch
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [38]:
#score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.7338891495453606

Coherence Score:  0.3434440566745277


In [39]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds='mmds')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.170655  0.277550       1        1  44.722336
17    -0.362028 -0.050683       2        1  21.437178
3     -0.047162  0.412485       3        1   7.311320
0     -0.425498  0.046607       4        1   4.859594
2     -0.294061  0.192834       5        1   4.583370
10    -0.173604  0.128277       6        1   3.543908
9      0.248406  0.242814       7        1   3.370210
1     -0.161784 -0.197041       8        1   1.916579
18     0.109362  0.226205       9        1   1.835350
7      0.297143  0.045903      10        1   1.568149
12    -0.061126 -0.308785      11        1   1.315439
13     0.172652 -0.221011      12        1   0.821552
19     0.067116 -0.232556      13        1   0.739312
5      0.235831 -0.104779      14        1   0.611256
15     0.103140 -0.156740      15        1   0.542540
8      0.170198 -0.047142      16        1   0.273487
11     0.082915 -0.030985      17        1   0.190193
6      0.087006 -0.075836      18        1   0.174849
16     0.049960 -0.081206      19        1   0.101422
4      0.072192 -0.065912      20        1   0.081957, topic_info=          Term        Freq       Total Category  logprob  loglift
131       item  360.000000  360.000000  Default  30.0000  30.0000
169     report  168.000000  168.000000  Default  29.0000  29.0000
51   inventory  274.000000  274.000000  Default  28.0000  28.0000
2      account  327.000000  327.000000  Default  27.0000  27.0000
380      erick  120.000000  120.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
39        food    0.007965   46.332409  Topic20  -7.5364  -1.5618
574    weekend    0.007965    1.224959  Topic20  -7.5364   2.0712
65      moment    0.007965   92.351284  Topic20  -7.5364  -2.2515
114    archive    0.007965  162.346586  Topic20  -7.5364  -2.8157
78     resolve    0.007965   15.973693  Topic20  -7.5364  -0.4969

[1019 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.606449        able
0         2  0.385047        able
112       1  0.927966  absolutely
1         1  0.077587      access
1         2  0.678885      access
...     ...       ...         ...
301       7  0.355483       wrong
301       9  0.059247       wrong
166       1  0.576996         yet
166       2  0.432747         yet
779       7  0.942195        zeke

[1095 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 18, 4, 1, 3, 11, 10, 2, 19, 8, 13, 14, 20, 6, 16, 9, 12, 7, 17, 5])